In [1]:
import pandas as pd

pd.set_option('max_colwidth', 1000)
pd.set_option('display.max_rows', None)

In [2]:
# open csv files
# Google Play data
df_gp = pd.read_csv('<Google-Play-downloads-file>.csv')
# App Store data
df_as = pd.read_csv('<App-Store-downloads-file>.csv')

In [ ]:
# drop rows with index 0, 1, 2, 3 in dataframe df_as
df_as.drop(df_as.index[[0, 1, 2, 3]], inplace=True)
df_as = df_as.reset_index()
# parse data to new column parsed before point
df_as['<app-name>_parsed'] = df_as['<app-name>'].str.split('.').str[0]
df_as

In [10]:
# parse data in column with downloads and drop all symbols except numbers
df_gp['User acquisition (New users, All events, Per interval, Daily): <Country>'] = df_gp['User acquisition (New users, All events, Per interval, Daily): <Country>'].str.replace('[^0-9]', '')

In [12]:
# convert string 6/30/22 to date format YYYY-MM-DD
df_as['date'] = pd.to_datetime(df_as['Name'], format='%m/%d/%y').reset_index(drop=True)

# convert string Jan 01, 2022 to date format YYYY-MM-DD
df_gp['date'] = pd.to_datetime(df_gp['Date'], format='%b %d, %Y')

In [15]:
# rename columns in dataframes
df_as.rename(columns={
    '<app-name>_parsed': 'app_store_downloads'}, inplace=True)

df_gp.rename(columns={
    'User acquisition (New users, All events, Per interval, Daily): <Country>': 'google_play_downloads'}, inplace=True)

# drop columns
df_as.drop(columns=['Name', '<app-name>', 'index'], inplace=True)
df_gp.drop(columns=['Date', 'Notes'], inplace=True)

In [20]:
# Downloads
# merge dataframes df_gp and df_as on date
df_downloads = pd.merge(df_gp, df_as, on='date', how='outer')

In [22]:
# convert column with strings to integer in dataframe df_downloads
df_downloads['google_play_downloads'] = df_downloads['google_play_downloads'].astype(int)
df_downloads['app_store_downloads'] = df_downloads['app_store_downloads'].astype(int)

In [ ]:
df_downloads

In [27]:
# take date column and create new column with date format YYYY-MM
df_downloads['date_month'] = df_downloads['date'].dt.strftime('%Y-%m')

In [29]:
df_downloads_by_month = df_downloads.groupby(['date_month']).agg({'google_play_downloads': 'sum', 'app_store_downloads': 'sum'}).reset_index()

In [31]:
# export to csv file
df_downloads_by_month.to_csv('df_downloads_by_month.csv', index=True)
df_downloads.to_csv('df_downloads_by_days.csv', index=True)